<a href="https://colab.research.google.com/github/RoyElkabetz/Text-Summarization-with-Deep-Learning/blob/main/Text_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## uncomment only if running from google.colab
# clone the git reposetory
!git clone https://github.com/RoyElkabetz/Text-Summarization-with-Deep-Learning
# add path to .py files for import
import sys
sys.path.insert(1, "/content/Text-Summarization-with-Deep-Learning")

Cloning into 'Text-Summarization-with-Deep-Learning'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 53 (delta 28), reused 6 (delta 0), pack-reused 0
Unpacking objects: 100% (53/53), done.


In [21]:
%matplotlib inline
import time

import torch
# from torchtext.datasets import IMDB as the_dataset
from torchtext.datasets import AG_NEWS as the_dataset
import torchtext.data as data
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.functional import to_map_style_dataset
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
from torch import nn




print(f'torch {torch.__version__}')
print('Device properties:')
if torch.cuda.is_available():
    device = torch.device("cuda")
    gpu_data = torch.cuda.get_device_properties(0)
    gpu_name = gpu_data.name
    gpu_mem  = f'{gpu_data.total_memory * 1e-9:.02f} Gb'
    print(f'GPU: {gpu_name}\nMemory: {gpu_mem}')
else:
    device = torch.device("cpu")
    print('CPU')

torch 1.9.0+cu102
Device properties:
CPU


In [22]:
# from torchtext.utils import (
#     download_from_url,
# )
# from torchtext.data.datasets_utils import (
#     _RawTextIterableDataset,
#     _wrap_split_argument,
#     _add_docstring_header,
#     _create_dataset_directory,
#     _create_data_from_csv,
# )
# import os

# URL = {
#     'train': "https://raw.githubusercontent.com/mhjabreel/CharCnn_Keras/master/data/ag_news_csv/train.csv",
#     'test': "https://raw.githubusercontent.com/mhjabreel/CharCnn_Keras/master/data/ag_news_csv/test.csv",
# }

# MD5 = {
#     'train': "b1a00f826fdfbd249f79597b59e1dc12",
#     'test': "d52ea96a97a2d943681189a97654912d",
# }

# NUM_LINES = {
#     'train': 120000,
#     'test': 7600,
# }

# DATASET_NAME = "AG_NEWS"


# # [docs]@_add_docstring_header(num_lines=NUM_LINES, num_classes=4)
# # @_create_dataset_directory(dataset_name=DATASET_NAME)
# # @_wrap_split_argument(('train', 'test'))
# def the_dataset(split):
#     root = '/content/dataset'
#     path = download_from_url(URL[split], root=root,
#                              path=os.path.join(root, split + ".csv"),
#                              hash_value=MD5[split],
#                              hash_type='md5')
#     return _RawTextIterableDataset(DATASET_NAME, NUM_LINES[split],
#                                    _create_data_from_csv(path))

In [23]:
tokenizer = get_tokenizer('basic_english')
train_iter = the_dataset(split='train')

def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>", "<sos>", "<eos>"])
vocab.set_default_index(vocab["<unk>"])

In [24]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x) - 1  # for {1, 2, ...} int classes
# label_pipeline = lambda x: 0 if x=='neg' else 1  # for IMDB

In [25]:
train_iter = the_dataset(split='train')
n_samples = len(train_iter)
random_list = torch.randint(0, n_samples - 1, (4, ))
labels = []
for i, (label, text) in enumerate(train_iter):
    labels.append(label)
    if i in random_list:
        print(f'Label: {label_pipeline(label)}')
        print(f'Text: {text}')
        print(f'Split: {tokenizer(text)}')
        print(f'Tokens: {text_pipeline(text)}\n')
print('Number of classes: {}'.format(len(set(labels))))
print('Number of samples: {}'.format(n_samples))

Label: 0
Text: Marines Say Up to 100 Insurgents Killed in Falluja  BAGHDAD (Reuters) - U.S. Marines said on Tuesday that as  many as 100 insurgents had been killed in fighting in the  hotbed Iraqi town of Falluja.
Split: ['marines', 'say', 'up', 'to', '100', 'insurgents', 'killed', 'in', 'falluja', 'baghdad', '(', 'reuters', ')', '-', 'u', '.', 's', '.', 'marines', 'said', 'on', 'tuesday', 'that', 'as', 'many', 'as', '100', 'insurgents', 'had', 'been', 'killed', 'in', 'fighting', 'in', 'the', 'hotbed', 'iraqi', 'town', 'of', 'falluja', '.']
Tokens: [2502, 235, 45, 6, 752, 1140, 144, 9, 1759, 241, 15, 29, 16, 17, 53, 3, 11, 3, 2502, 28, 12, 59, 19, 21, 308, 21, 752, 1140, 88, 83, 144, 9, 831, 9, 4, 17321, 210, 997, 8, 1759, 3]

Label: 0
Text: Iraq PM Adamant Jan. Election Will Be Held Iraqi Prime Minister Iyad Allawi said Sunday elections would be held on schedule in January despite a surge in violence and more than a score of hostages under threat of death from insurgents.
Split: ['ira

In [26]:
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)    

train_iter = the_dataset(split='train')
dataloader = DataLoader(train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch)

In [27]:
class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.activation = nn.ReLU()
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        x = self.fc(embedded)
        output = self.activation(x)
        return output

In [28]:
train_iter = the_dataset(split='train')
num_class = len(set([label for (label, text) in train_iter]))
vocab_size = len(vocab)
emsize = 64
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

In [29]:
def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predited_label = model(text, offsets)
        loss = criterion(predited_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predited_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predited_label = model(text, offsets)
            loss = criterion(predited_label, label)
            total_acc += (predited_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [30]:
# Hyperparameters
EPOCHS = 10 # epoch
LR = 5  # learning rate
BATCH_SIZE = 64 # batch size for training
  
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
train_iter, test_iter = the_dataset()
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = \
    random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

test.csv: 1.86MB [00:00, 35.3MB/s]                  


| epoch   1 |   500/ 1782 batches | accuracy    0.505
| epoch   1 |  1000/ 1782 batches | accuracy    0.645
| epoch   1 |  1500/ 1782 batches | accuracy    0.754
-----------------------------------------------------------
| end of epoch   1 | time: 14.69s | valid accuracy    0.825 
-----------------------------------------------------------
| epoch   2 |   500/ 1782 batches | accuracy    0.851
| epoch   2 |  1000/ 1782 batches | accuracy    0.865
| epoch   2 |  1500/ 1782 batches | accuracy    0.893
-----------------------------------------------------------
| end of epoch   2 | time: 14.41s | valid accuracy    0.901 
-----------------------------------------------------------
| epoch   3 |   500/ 1782 batches | accuracy    0.911
| epoch   3 |  1000/ 1782 batches | accuracy    0.910
| epoch   3 |  1500/ 1782 batches | accuracy    0.911
-----------------------------------------------------------
| end of epoch   3 | time: 14.57s | valid accuracy    0.904 
-------------------------------

In [31]:
print('Checking the results of test dataset.')
accu_test = evaluate(test_dataloader)
print('test accuracy {:8.3f}'.format(accu_test))

Checking the results of test dataset.
test accuracy    0.907


In [32]:
ag_news_label = {1: "World",
                 2: "Sports",
                 3: "Business",
                 4: "Sci/Tec"}

def predict(text, text_pipeline):
    with torch.no_grad():
        text = torch.tensor(text_pipeline(text))
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item() + 1

ex_text_str = "MEMPHIS, Tenn. – Four days ago, Jon Rahm was \
    enduring the season’s worst weather conditions on Sunday at The \
    Open on his way to a closing 75 at Royal Portrush, which \
    considering the wind and the rain was a respectable showing. \
    Thursday’s first round at the WGC-FedEx St. Jude Invitational \
    was another story. With temperatures in the mid-80s and hardly any \
    wind, the Spaniard was 13 strokes better in a flawless round. \
    Thanks to his best putting performance on the PGA Tour, Rahm \
    finished with an 8-under 62 for a three-stroke lead, which \
    was even more impressive considering he’d never played the \
    front nine at TPC Southwind."

model = model.to("cpu")

print("This is a %s news" %ag_news_label[predict(ex_text_str, text_pipeline)])

This is a Sports news
